## Contexte

LangChain permet de créer des applications utilisant des LLM.
Un souci récurrent de ce type d'application est le manque de fiabilité et de contrôle. C'est là qu'intervient [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview): une application permettant davantage de configuration et de supervision sur les systèmes agentiques.

## Structure du cours

Dans chaque module est intégré des cas pratiques en format Notebook et un répertoire `studio`, qui permet une visualisation des graphes dans [LangSmith Studio](https://docs.langchain.com/langsmith/quick-start-studio).

## Configuration

Avant de commencer: suivre les instructions du fichier `README.md`.

## Modèles conversationnel

On utilisera [ChatOpenAI](https://docs.langchain.com/oss/python/integrations/chat/openai) par défaut. Il est alors nécessaire de définir la variable `OPENAI_API_KEY`.
> D'autres modèles sont pris en charges, voir la [doc](https://docs.langchain.com/oss/python/integrations/chat).

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community langchain-tavily


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

Il existe [quelques paramètres standard](https://docs.langchain.com/oss/python/langchain/models#parameters) que nous pouvons définir avec les modèles conversationnels
 notamment:

- `model` : le nom du modèle
- `temperature` : la température d'échantillonnage

Plus la température est basse (proche de 0), moins les réponses seront variables et vice-versa lorsque l'on s'approche de 1.

## Exemple d'utilisation

In [3]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)

In [4]:
from langchain_core.messages import HumanMessage

# message initial
msg = HumanMessage(content="Hello world", name="Lance")

# liste de messages
messages = [msg]

# appel du modèle
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CYznmrtvLVE4MlDrvTQCwVwSL68c3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d7434659-8b5c-4d77-baa3-e69efc9273a8-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

On peut également appeler le modèle directement en lui passant une chaîne de caractères.

In [5]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_65564d8ba5', 'id': 'chatcmpl-CYznnbvnC28O3lEH83kcSCWZrU718', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--eb86f8b5-79b6-497d-9b33-72126568efb4-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

> L'interface n'est pas dépendente du modèle. Ce qui facilite les changements de fournisseurs.


## Outil de recherche

Pour le module 3, on utilisera [Tavily](https://tavily.com/), un moteur de recherche adapté aux LLM. Une offre d'essai gratuit est incluse.

In [6]:
_set_env("TAVILY_API_KEY")

In [7]:
from langchain_tavily import TavilySearch

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "What is Onepoint (Paris)?"})
search_docs = data.get("results", data)

In [8]:
search_docs

[{'url': 'https://www.officelovin.com/2018/02/look-inside-onepoints-modern-paris-office/',
  'title': "A Look Inside Onepoint's Modern Paris Office - Officelovin'",
  'content': '# A Look Inside Onepoint’s Modern Paris Office Onepoint, a global software company that helps companies develop their architecture, networks, systems and tools to increase their daily performance, hired interior design firm Archimage to design their new offices in Paris, France. > “One Point had its own vision of a place of work. That’s why only 50% of the place (3400 square meters) is composed with desks, and the other part with for relaxation spaces, kitchen and even with a sport room. They used a vegetal wall, Lego plates, metallic facing, differents wallpapers and tailored spaces like slides.” * ### A Tour of Bredin Prat’s Elegant Paris Office * ### A Look Inside Wall AG’s Berlin Office',
  'score': 0.767638,
  'raw_content': None},
 {'url': 'https://tracxn.com/d/companies/onepoint/__E26OvzL3xwkHDpRTBkpqy3